In [1]:
# Importing all neccessary libraries
import os
import pandas as pd
import torch
from torchvision import transforms

from PIL import Image
from torch.utils.data import DataLoader, random_split, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
import torchvision


import wandb
wandb.login(key= "d191b9506b59d660fca198916a51389adcf4f7ec")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
# Methods for chosing new parameters for the model
sweep_config = {
    'method': 'random'
    }

In [3]:
# Metrics for the model
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [4]:
# Parameters for the model
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd', 'rmsprop', 'adadelta', 'adagrad']
        },
#     'fc_layer_size': {
#         'values': [64, 128, 256, 512]
#         },
    'dropout': {
          'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [5]:
parameters_dict.update({
    'epochs': {
        'value': 12}
    })

In [6]:
# Additional parameters for the model
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 16,
        'max': 32,
      }
    })

In [7]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 32,
                               'min': 16,
                               'q': 8},
                'dropout': {'values': [0.1, 0.2, 0.3, 0.4, 0.5]},
                'epochs': {'value': 12},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam',
                                         'sgd',
                                         'rmsprop',
                                         'adadelta',
                                         'adagrad']}}}


In [8]:
sweep_id = wandb.sweep(sweep_config, project="Image Classification", entity = "ariharasudhan-muthusami")

Create sweep with ID: 9g2mjlya
Sweep URL: https://wandb.ai/ariharasudhan-muthusami/Image%20Classification/sweeps/9g2mjlya


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
train_csv = pd.read_csv('/kaggle/input/opencv-pytorch-dl-course-classification/train.csv')
train_data = pd.DataFrame(train_csv)
print("train_data info: ", train_data['class'].value_counts())

train_data info:  chapati        862
nyamachoma     784
bhaji          632
ugali          628
mandazi        620
kachumbari     494
matoke         483
githeri        479
masalachips    438
sukumawiki     402
pilau          329
mukimo         212
kukuchoma      173
Name: class, dtype: int64


In [11]:
# Label Encoder function
def label_encoder(df):
    if df["class"] == 'githeri' :
        return 0
    
    elif df["class"] == 'ugali' :
        return 1
    
    elif df["class"] == 'kachumbari' :
        return 2
    
    elif df["class"] == 'matoke':
        return 3
    
    elif df["class"] == 'sukumawiki':
        return 4
    
    elif df['class'] == 'bhaji':
        return 5
    
    elif df['class'] == 'nyamachoma':
        return 6
    
    elif df['class'] == 'mandazi':
        return 7
    
    elif df['class'] == 'pilau':
        return 8
    
    elif df['class'] == 'chapati':
        return 9
    
    elif df['class'] == 'kukuchoma':
        return 10
    
    elif df['class'] == 'masalachips':
        return 11
    
    elif df['class'] == 'mukimo':
        return 12

In [12]:
train_data['labels'] = train_data.apply(label_encoder, axis = 1)
train_data["id"] = train_data["id"].astype(str)
print(train_data)

                        id       class  labels
0     14278962987112149800     githeri       0
1     13190220095752321996       ugali       1
2     10431803432626641638  kachumbari       2
3      4222441716327528413     githeri       0
4      2547906925836120627      matoke       3
...                    ...         ...     ...
6531   9491780363201908054      matoke       3
6532    938716700684692863     chapati       9
6533  12487998559342412135       ugali       1
6534   3466916091782561952  kachumbari       2
6535   4410319397208590457     chapati       9

[6536 rows x 3 columns]


In [13]:
train_data.to_csv("train.csv")


In [14]:
# Custom dataloader function
class Kenyafood_data(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations['id'])
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, str(self.annotations['id'][index])+'.jpg')
        image = Image.open(img_path)
        #image = cv2.imread(img_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        y_label = torch.tensor(int(self.annotations['labels'][index]))
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, y_label

In [15]:
transform =  transforms.Compose(
    [transforms.Resize((224,224)),
      transforms.RandomHorizontalFlip(p = 0.5),
      transforms.RandomRotation(degrees = 45),
      transforms.RandomAutocontrast(p = 0.5),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
      ])
    
imageloader = Kenyafood_data(csv_file = "/kaggle/working/train.csv", root_dir = "/kaggle/input/opencv-pytorch-dl-course-classification/images/images",
                                          transform = transform)
train_sampler, val_sampler =  random_split(imageloader, [4536,2000]) 

In [16]:
def build_dataset(batch_size):
  
  loader = DataLoader(val_sampler, batch_size=batch_size)
  #print(next(iter(loader)))

  return loader

In [17]:
# Resnet34 model
# def build_network():
#     res_model = models.resnet34(pretrained=True)
#     res_model.fc = nn.Linear(512, 13)
#     network = res_model

#     return network.to(device)

# Efficientnet model
# def build_network():
#     res_model = models.efficientnet_b4(pretrained = True)
#     res_model.classifier[1] = nn.Linear(in_features=1792, out_features=13)
#     network = res_model

#     return network.to(device)


In [18]:
def build_network(dropout):
    # res_model = models.resnet18(pretrained=True)
    # res_model.fc = nn.Sequential(nn.Dropout(p=dropout), nn.Linear(512, 13))
    
    res_model = models.efficientnet_b0(pretrained = True)
    res_model.classifier = nn.Sequential(nn.Dropout(p = dropout),nn.Linear(in_features=1280, out_features=13)) 

    for param in res_model.parameters():
        param.requires_grad = True
    
    network = res_model

    return network.to(device)

In [19]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    elif optimizer == "rmsprop":
        optimizer = optim.RMSprop(network.parameters(), 
                               lr = learning_rate)
        
    elif optimizer == "adadelta":
        optimizer = optim.Adadelta(network.parameters(),
                               lr = learning_rate)
    elif optimizer == "adagrad":
        optimizer = optim.Adagrad(network.parameters(),
                               lr = learning_rate)
    
    
    return optimizer


In [20]:
criterion = nn.CrossEntropyLoss()

def train_epoch(network, loader, optimizer):

    wandb.watch(network, criterion, log="all", log_freq=100)
    network.train()
    running_loss = 0
    
    for _, (data, target) in enumerate(loader,0):
        data = data.to(device)
       # print('data shape:',data.shape)
        target = target.to(device)
        #print('targets and its shape are :', target, target.shape)
        optimizer.zero_grad()

        # Forward pass
        prediction = network(data)
       # print('predictions and its shape are: ', prediction, prediction.shape)

        loss =  criterion(prediction, target)
        #print("loss after cross entropy loss ",loss)
 
        # Backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        wandb.log({"batch loss": loss.item()})
    return running_loss / len(loader)

In [21]:

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(dropout=config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

In [22]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: wivcm304 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 12
wandb: 	learning_rate: 0.05121208101656545
wandb: 	optimizer: sgd
wandb: Currently logged in as: ariharasudhan-muthusami. Use `wandb login --relogin` to force relogin


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

batch loss,▇▄█▆█▆▇▅▄▅▅▅▄▄▄▅▄▅▄▄▃▃▃▄▃▃▃▂▃▂▂▂▁▃▂▁▁▂▁▂
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,██▆▅▄▃▃▂▂▂▁▁
batch loss,0.63403
epoch,11
loss,0.59689


wandb: Agent Starting Run: 79o780fz with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 12
wandb: 	learning_rate: 0.080144167918732
wandb: 	optimizer: sgd


...

batch loss,▇▇▇▇█▅▇▆▄▅▆▆▅▅▄▅▃▅▄▃▄▃▃▄▃▃▃▃▂▂▂▃▂▃▂▁▂▂▃▂
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▇▅▅▄▃▃▃▂▂▁▂
batch loss,0.34206
epoch,11
loss,0.84759


wandb: Agent Starting Run: z0if554k with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.4
wandb: 	epochs: 12
wandb: 	learning_rate: 0.041224211694680706
wandb: 	optimizer: adam


batch loss,█▂▂▂▃▂▂▃▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▃▃▂▄▂▃▃▂▂▁▂▂▁▁▃▂▂
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▃▂▂▂▂▂▂▂▁▁▁
batch loss,2.28719
epoch,11
loss,2.52135


wandb: Agent Starting Run: xe52ptf4 with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.5
wandb: 	epochs: 12
wandb: 	learning_rate: 0.04799744841415576
wandb: 	optimizer: adagrad


batch loss,█▄▄▅▃▄▄▄▄▄▃▃▃▄▃▂▄▂▂▃▂▂▃▃▂▂▄▃▄▄▂▃▂▂▂▁▁▃▂▂
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▅▅▄▄▃▃▂▂▂▁▁
batch loss,1.68579
epoch,11
loss,1.63986
